# IMDb Scraper

Workflow: 5 

Goal: Start with TMDB ids and get all data from IMDB.

Result: The file ```imdb_movie.csv``` is created.

In [1]:
import os
import time
import pandas as PD

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# can't reload script in a different folder

import sys
sys.path.append('../models')
import imdb_scraper as MB 

## Scrape One Movie

In [4]:
scraper = MB.ImdbScraper()
scraper.open_imdb()

<selenium.webdriver.firefox.webdriver.WebDriver (session="26ffc363-d55e-1a4a-a5bb-d5399154025c")>


In [5]:
title = 'Kaaliyan'
year = '2017'
# scraper.search_any(title)

In [6]:
# scraper.goto_movie_result(title, year)

In [7]:
# movie_dx = scraper.get_movie_data()
# movie_dx

In [8]:
movie_dx = scraper.get_movie_by_id('tt3170504')
movie_dx 

{'imdb_id': 'tt3170504',
 'title': 'Last Love Lost',
 'original_title': None,
 'year': '2015',
 'rating': 'Not Rated',
 'companies': None,
 'country': None,
 'language': None,
 'duration': 83,
 'directors': 'Jahmar Hill',
 'writers': 'Jahmar Hill',
 'actors': 'Jahmar Hill, Toni Belafonte, Talli Clemons',
 'genres': 'Drama',
 'synopsis': 'Antonio (Jahmar Hill) is awakened from a coma he was in for a year and a half after being shot by his father while trying to break up a domestic dispute between him and his mother. When ',
 'budget': None,
 'gross_us': None,
 'gross_worldwide': None,
 'score': '7.5',
 'votes': '15'}

In [9]:
scraper.close()

## Scrape Based on TMDB

In [10]:
INDEX_FILE = '../data/moviedb_movie.csv'
index_df = PD.read_csv(INDEX_FILE, dtype={'title': str, 'year': str})
index_df.head(3)
index_df.info()

,title,original_title,year,companies,country,language,run_time,crew,cast,poster,genres,collection,synopsis,budget,gross,score,votes,tmdb_id,imdb_id
0,1,1,2013,"Exclusive Media, Flat-Out Films",United States of America,English,112.0,"Paul Crowder, Mark Monroe, Michael Shevloff","Niki Lauda, Michael Schumacher, Lewis Hamilton",/4uIPXX8DjTsCzUAdtMKHTpojYLq.jpg,Documentary,NaN,Set in the golden era of Grand Prix Racing '1'...,NaN,NaN,7.4,59,217316,tt2518788
1,"10,000 BC","10,000 BC",2008,"Centropolis Entertainment, Legendary Entertain...",United States of America,English,109.0,"Roland Emmerich, Sarah Bradshaw, Tom Karnowski","Steven Strait, Camilla Belle, Cliff Curtis",/rnGR3EHkL4ryhQd50XBrtRrV8nq.jpg,"Adventure, Action, Drama, Fantasy",NaN,A prehistoric epic that follows a young mammot...,105000000.0,266000000.0,5.3,1766,7840,tt0443649
2,1000 Rupee Note,Ek Hazarachi Note,2016,NaN,India,Marathi (Marāṭhī),89.0,"Shrihari Sathe, Shrikant Bojewar",Sandeep Pathak,/pNNxwXAReV4kh7TCZGqBrl9I72v.jpg,Drama,NaN,Poor Parobudhi receives a thousand rupee note ...,NaN,NaN,6.9,7,318654,tt2937158


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17668 entries, 0 to 17667
Data columns (total 19 columns):
title             17668 non-null object
original_title    17668 non-null object
year              17668 non-null object
companies         11589 non-null object
country           13571 non-null object
language          17667 non-null object
run_time          16507 non-null float64
crew              16720 non-null object
cast              16002 non-null object
poster            17296 non-null object
genres            16109 non-null object
collection        1189 non-null object
synopsis          17537 non-null object
budget            2863 non-null float64
gross             1908 non-null float64
score             17668 non-null float64
votes             17668 non-null int64
tmdb_id           17668 non-null int64
imdb_id           17456 non-null object
dtypes: float64(4), int64(2), object(13)
memory usage: 2.6+ MB


In [11]:
# create jobs

index_1_df = index_df[0:5349]
index_2_df = index_df[5350:9999]
index_3_df = index_df[10000:13999]
index_4_df = index_df[14000:]

index_1_df.shape
index_2_df.shape
index_3_df.shape
index_4_df.shape

(5349, 19)

(4649, 19)

(3999, 19)

(3668, 19)

In [12]:
# investigate errors

error_titles = ['13 Times Evil', 'There Is a New World Somewhere', 'The Scarlet and the Black', 'The Sleeping Tiger', 
                'The Untold Story', 'Zombiegeddon', 'To Love The Soul Of A Woman', 'Tree Man']
index_df.loc[index_df['title'].isin(error_titles)][['title', 'year', 'imdb_id']]

,title,year,imdb_id
56,13 Times Evil,2016,NaN
15415,There Is a New World Somewhere,2015,tt3281502
15524,The Scarlet and the Black,1983,tt0086251
15634,The Sleeping Tiger,1954,tt0047505
15910,The Untold Story,1993,tt0103743
16288,To Love The Soul Of A Woman,2017,NaN
16452,Tree Man,2016,tt3246684
17642,Zombiegeddon,2003,tt0316946


In [13]:
#index_df.tail(20)['title']

In [14]:
scraper = MB.ImdbScraper()
scraper.open_imdb()
movie_ls = []
t0 = time.time()

for idx, row in index_4_df.iterrows():
    title = row['title']
    year = row['year']
    imdb_id = row['imdb_id']
    movie_dx = None

    try:
        movie_dx = scraper.get_movie_by_id(imdb_id)
    except Exception as ex:
        print('')
        print(f'Error 1: {title} {year} {imdb_id}')
        print(ex)

    if not movie_dx:
        try:
            scraper.open_imdb()
            scraper.search_any(title)
            found = scraper.goto_movie_result(title, year)
            if found:
                print(f'found by title: {title} {year} {imdb_id}')
                movie_dx = scraper.get_movie_data()
            else:
                print(f'NOT found by title: {title} {year}')
        except Exception as ex:
            print(f'Error 2: {title} {year} {imdb_id}')
            print(ex)

    if movie_dx:
        movie_ls.append(movie_dx)

scraper.close()
t1 = time.time()
print(f'movies: {len(movie_ls)}')
print(f'time: {(t1-t0)/60/60:.2f} hrs')

<selenium.webdriver.firefox.webdriver.WebDriver (session="28eef395-5ff2-2741-82ca-edbccd4ee2c6")>

Error 1: The Fairies: Fairy Beach 2007 nan
Message: Unable to locate element: .title_wrapper

found by title: The Fairies: Fairy Beach 2007 nan

Error 1: The Fence (La Barda) 2010 tt1853700
get_movie_data(): title error: tt1853700
NOT found by title: The Fence (La Barda) 2010

Error 1: The GiggleBellies Sweet Songs & Lullabies 2013 nan
Message: Unable to locate element: .title_wrapper

Error 2: The GiggleBellies Sweet Songs & Lullabies 2013 nan
Message: Unable to locate element: //*[h3/text()="Titles"]


Error 1: The Gray Seasons 2013 nan
Message: Unable to locate element: .title_wrapper

NOT found by title: The Gray Seasons 2013

Error 1: The Great Buddha+ 2017 tt7010412
get_movie_data(): title error: tt7010412
NOT found by title: The Great Buddha+ 2017

Error 1: The Green Planet 2012 nan
Message: Unable to locate element: .title_wrapper

found by title: The Green Planet 2012 nan

Error 

found by title: VA: The Human Cost of War 2017 nan

Error 1: Walk By Faith 2014 nan
Message: Unable to locate element: .title_wrapper

found by title: Walk By Faith 2014 nan

Error 1: What Makes a Psychopath? 2017 tt7314990
get_movie_data(): title error: tt7314990
NOT found by title: What Makes a Psychopath? 2017

Error 1: When They Were Young 2011 nan
Message: Unable to locate element: .title_wrapper

NOT found by title: When They Were Young 2011

Error 1: Who Is Harry Nilsson (And Why Is Everybody Talkin' About Him?) 2010 tt0756727
get_movie_data(): title error: tt0756727
NOT found by title: Who Is Harry Nilsson (And Why Is Everybody Talkin' About Him?) 2010

Error 1: Who Put The Klan Into Ku Klux Klan 2018 nan
Message: Unable to locate element: .title_wrapper

Error 2: Who Put The Klan Into Ku Klux Klan 2018 nan
Message: Unable to locate element: //*[h3/text()="Titles"]


Error 1: Wild Kratts: Creepy Creatures 2018 nan
Message: Unable to locate element: .title_wrapper

NOT found by 

In [15]:
# projected time

#print(f'projected time: {(t1-t0)/60/60*18450/index_df.shape[0]:.2f} hrs')

In [16]:
movie_df = PD.DataFrame(movie_ls)
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3623 entries, 0 to 3622
Data columns (total 19 columns):
imdb_id            3623 non-null object
title              3623 non-null object
original_title     556 non-null object
year               3623 non-null object
rating             2796 non-null object
companies          3329 non-null object
country            3607 non-null object
language           3569 non-null object
duration           3481 non-null float64
directors          3601 non-null object
writers            3295 non-null object
actors             3435 non-null object
genres             3623 non-null object
synopsis           3623 non-null object
budget             1113 non-null object
gross_us           774 non-null object
gross_worldwide    1202 non-null object
score              3599 non-null object
votes              3599 non-null object
dtypes: float64(1), object(18)
memory usage: 537.9+ KB


In [17]:
save_path = r'../data/imdb_mov_4.csv'
movie_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

'1.35 mb'

## Combine Job Files

In [18]:
#stop = 0 0
imdb_1_df = PD.read_csv('../data/imdb_mov_1.csv')
imdb_2_df = PD.read_csv('../data/imdb_mov_2.csv')
imdb_3_df = PD.read_csv('../data/imdb_mov_3.csv')
imdb_4_df = PD.read_csv('../data/imdb_mov_4.csv')

imdb_1_df.shape
imdb_2_df.shape
imdb_3_df.shape
imdb_4_df.shape

(5358, 19)

(4590, 19)

(3956, 19)

(3623, 19)

In [19]:
movie_df = PD.concat([imdb_1_df, imdb_2_df, imdb_3_df, imdb_4_df])
movie_df.head(3)
movie_df.info()

,imdb_id,title,original_title,year,rating,companies,country,language,duration,directors,writers,actors,genres,synopsis,budget,gross_us,gross_worldwide,score,votes
0,tt2518788,1,NaN,2013,Not Rated,"Diamond Docs, Exclusive Media Group",USA,English,112.0,Paul Crowder,Mark Monroe,"Michael Fassbender, Niki Lauda, Lewis Hamilton","Documentary, History, Sport",Set in the golden era of Grand Prix Racing '1'...,NaN,NaN,NaN,8.0,3535.0
1,tt0443649,"10,000 BC",NaN,2008,PG-13,"Warner Bros., Legendary Entertainment",USA,English,109.0,Roland Emmerich,"Roland Emmerich, Harald Kloser","Camilla Belle, Steven Strait, Marco Khan","Action, Adventure, Drama","In the prehistoric past, D'Leh is a mammoth hu...",105000000,9.47842e+07,269784201,5.1,121431.0
2,tt2937158,Ek Hazarachi Note,NaN,2014,Not Rated,Infinitum Productions,India,Marathi,89.0,Shrihari Sathe,Shrikant Bojewar,"Devendra Gaikwad, Usha Naik, Pooja Nayak",Drama,"In this exploration of money and conscience, a...",NaN,2404,2404,7.3,309.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17527 entries, 0 to 3622
Data columns (total 19 columns):
imdb_id            17527 non-null object
title              17527 non-null object
original_title     2316 non-null object
year               17527 non-null int64
rating             13288 non-null object
companies          16175 non-null object
country            17444 non-null object
language           17288 non-null object
duration           16674 non-null float64
directors          17420 non-null object
writers            15969 non-null object
actors             16446 non-null object
genres             17517 non-null object
synopsis           17432 non-null object
budget             5198 non-null object
gross_us           3393 non-null object
gross_worldwide    5419 non-null object
score              17406 non-null float64
votes              17406 non-null float64
dtypes: float64(3), int64(1), object(15)
memory usage: 2.7+ MB


In [20]:
# remove duplicates

movie_df = movie_df.drop_duplicates()
movie_df = movie_df.drop_duplicates(subset=['imdb_id'])
movie_df.shape

(17398, 19)

In [21]:
# fix up column types

movie_df['duration'] = movie_df['duration'].astype('Int64')
movie_df['votes'] = movie_df['votes'].astype('Int64')

In [22]:
save_path = r'../data/imdb_movie.csv'
movie_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

'6.38 mb'